In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/stat940project')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import random
import math
import torch.optim as optim
from tqdm.auto import tqdm, trange

In [6]:
from dataset import load_data, load_targets, load_adv_data, AdversarialDataset
from utils.augmentations import *
from utils.utils import set_seed, pcshow
from models import Pointnet, DGCNN
from defenses import SRS, SOR

In [7]:
set_seed(1)

In [8]:
parent_dir = '/content/drive/MyDrive/stat940project'

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')

else:
    device = torch.device('cpu')
    print('running on CPU')

running on GPU


In [10]:
def evaluate_samples(model, x, y, targets, transforms, batch_size=64, targeted=False):
    model.eval()

    testset = AdversarialDataset(x, y, targets, transforms=transforms)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    total = 0
    correct = 0

    with torch.no_grad():
        for i, data in enumerate(testloader):
            points, labels, targets = data
            points = points.to(device)
            labels = labels.to(device)
            targets = targets.to(device)
            points = points.transpose(1, 2).float()

            logits = model(points)
            if isinstance(logits, tuple):
                logits = logits[0]

            _, predicted = torch.max(logits.data, 1)
            total += labels.shape[0]
            if targeted:
                correct += (targets == predicted).sum().item()
            else:
                correct += (labels == predicted).sum().item()

        acc = 100 * (correct / total)
        return acc

In [11]:
def apply_defense(defender, x, y, targets, batch_size=64):
    all_defense_outputs = []

    for batch_idx in trange(0, len(x), batch_size):
        batch_pc = x[batch_idx:batch_idx + batch_size]
        batch_pc = torch.from_numpy(batch_pc)

        batch_pc = batch_pc.to(device)
        defense_output = defender.defense(batch_pc)

        if isinstance(defense_output, list) or isinstance(defense_output, tuple):
            defense_output = [
                pc.detach().cpu().numpy().astype(np.float32) for pc in defense_output
            ]

        else:
            defense_output = defense_output.detach().cpu().numpy().astype(np.float32)
            defense_output = [pc for pc in defense_output]

        all_defense_outputs += defense_output

    if isinstance(defender, SOR):
        all_defense_outputs = np.array(all_defense_outputs, dtype=object)
    else:
        all_defense_outputs = np.array(all_defense_outputs)

    return all_defense_outputs

In [13]:
attacks = ["FGM", "IFGM", "PGD", "DeepFool"]
models = ["Pointnet", "DGCNN"]
targeted = False

test_transforms = transforms.Compose(
    [
        Normalize(),
        ToTensor()
    ]
)
default_transforms = transforms.Compose(
    [
        ToTensor()
    ]
)


for attack in attacks:
    for model_name in models:

        if model_name == "Pointnet":
            model = Pointnet(device=device).to(device)
        else:
            model = DGCNN().to(device)

        checkpoint_path = os.path.join(parent_dir, f'checkpoints/{model_name.lower()}/best.pt')
        model.load_state_dict(torch.load(checkpoint_path, map_location=device)["model_state_dict"])

        data_path = os.path.join(parent_dir, 'data/ModelNet40')
        test_X, test_y = load_data(data_path, mode="test")
        test_targets = load_targets(data_path)

        test_acc = evaluate_samples(model, test_X, test_y, test_targets, test_transforms)
        print(f"Accuracy of {model_name} on test dataset is {test_acc:.2f}%.")

        adv_data_path = os.path.join(parent_dir, f'data/{attack}/{model_name}')
        adv_X, adv_y, adv_targets = load_adv_data(adv_data_path, targeted=targeted)

        adv_acc = evaluate_samples(model, adv_X, adv_y, adv_targets, default_transforms, targeted=targeted)
        print(f"Accuracy of {model_name} against {attack} is {adv_acc:.2f}%.")

        defenses = [SRS(), SOR()]
        defense_batch_sizes = [64, 1]

        for i in range(len(defenses)):
            defender = defenses[i]
            after_defense = apply_defense(defender, adv_X, adv_y, adv_targets)
            def_acc = evaluate_samples(model, after_defense, adv_y, adv_targets,
                                       default_transforms,
                                       batch_size=defense_batch_sizes[i],
                                       targeted=targeted)

            print(f"Accuracy of {model_name} against {attack} using {defender.__class__.__name__} defense is {def_acc:.2f}%.")


        print("---------------------------------------------------------------")

Accuracy of Pointnet on test dataset is 87.32%.
Accuracy of Pointnet against FGM is 41.90%.


  0%|          | 0/39 [00:00<?, ?it/s]

Accuracy of Pointnet against FGM using SRS defense is 50.12%.


  0%|          | 0/39 [00:00<?, ?it/s]

Accuracy of Pointnet against FGM using SOR defense is 58.67%.
---------------------------------------------------------------
Accuracy of DGCNN on test dataset is 90.84%.
Accuracy of DGCNN against FGM is 77.96%.


...

Accuracy of Pointnet against DeepFool using SOR defense is 6.32%.
---------------------------------------------------------------
Accuracy of DGCNN on test dataset is 90.84%.
Accuracy of DGCNN against DeepFool is 3.48%.


  0%|          | 0/39 [00:00<?, ?it/s]

Accuracy of DGCNN against DeepFool using SRS defense is 25.61%.


  0%|          | 0/39 [00:00<?, ?it/s]

Accuracy of DGCNN against DeepFool using SOR defense is 15.03%.
---------------------------------------------------------------
